In [1]:
# !pip install catboost

In [2]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn import metrics
from catboost import CatBoostClassifier

%matplotlib inline

The objective of this competition is to create a machine learning model to determine if a user will be active on Zindi in the next month. An active user is one that enters a competition, makes a submission or engages through the discussion forums. Just imagine, you are one of the data points in this challenge!

In [3]:
comments_df = pd.read_csv("Comments.csv")
competition_participation_df = pd.read_csv("CompetitionPartipation.csv")
competitions_df = pd.read_csv("Competitions.csv")
discussions_df = pd.read_csv("Discussions.csv")
sample_submission_df = pd.read_csv("SampleSubmission.csv")
submissions_df = pd.read_csv("Submissions.csv")
test_df = pd.read_csv("Test.csv")
train_df = pd.read_csv("Train.csv")
users_df = pd.read_csv("Users.csv")
variable_definitions_df = pd.read_csv("VariableDefinitions.csv")

In [4]:
variable_definitions_df[variable_definitions_df["VariableName"] == "FeatureY"].values

array([['FeatureY',
        'A category the user falls into. This field is masked.']],
      dtype=object)

In [5]:
sample_submission_df

,UserMonthYear,Target
0,ID_000VV0KM_1_4,NaN
1,ID_000VV0KM_2_4,NaN
2,ID_000VV0KM_3_4,NaN
3,ID_003OCIYO_1_4,NaN
4,ID_003OCIYO_2_4,NaN
...,...,...
65218,ID_ZZVPF22K_2_4,NaN
65219,ID_ZZVPF22K_3_4,NaN
65220,ID_ZZXDLYXB_1_4,NaN
65221,ID_ZZXDLYXB_2_4,NaN


In [6]:
comments_df

,UserID,CommentDate Year,CommentDate Month,CommentDate Day_of_week
0,ID_MVIB05DL,3,1,7
1,ID_MVIB05DL,3,1,7
2,ID_KBRFRAR9,3,1,7
3,ID_UDS6FRR8,3,1,7
4,ID_UDS6FRR8,3,1,7
...,...,...,...,...
11746,ID_9UP7X8IA,3,5,7
11747,ID_9MJ12JJR,3,5,4
11748,ID_0B9SK73T,3,6,5
11749,ID_0B9SK73T,3,6,4


In [7]:
competition_participation_df

,CompID,UserID,PublicRank,Successful Submission Count,CompPartCreated Year,CompPartCreated Month,CompPartCreated Day_of_week
0,ID_WMUF,ID_UWBBZ9OF,rank 11,count 10,3,1,4
1,ID_MPSN,ID_UWBBZ9OF,NaN,NaN,3,1,7
2,ID_WMUF,ID_1N5J2PGO,NaN,count 10,3,1,5
3,ID_7MLO,ID_1N5J2PGO,NaN,NaN,2,10,5
4,ID_2KEY,ID_1N5J2PGO,NaN,NaN,2,5,4
...,...,...,...,...,...,...,...
48560,ID_XYJZ,ID_5C4D0VO2,rank 11,count 10,3,7,6
48561,ID_XYJZ,ID_JRJZQB8S,rank 11,count 10,3,7,5
48562,ID_XYJZ,ID_H4FM7RBV,NaN,NaN,3,7,5
48563,ID_XYJZ,ID_C3QOEMU4,rank 11,count 10,3,7,5


In [8]:
competitions_df

,CompID,Country,FeatureA,FeatureB,FeatureC,Kind,Points Reward,FeatureD,SecretCode,SubmissionLimitPerDay,FeatureE,CompEndTime Year,CompEndTime Month,CompEndTime Day_of_week,CompStartTime Year,CompStartTime Month,CompStartTime Day_of_week
0,ID_WGZ2,ID_HWRH,[1],[14],1.0,1,27,1,0,100.0,[1],1,8.0,7.0,1,6,5
1,ID_G370,NaN,[1],[],2.0,1,52,2,0,10.0,[1],4,1.0,6.0,3,12,7
2,ID_R5HL,NaN,[1],[],3.0,1,126,2,0,10.0,[1],4,1.0,6.0,3,10,4
3,ID_Y6XI,ID_ARVG,[1],[],4.0,1,52,2,0,10.0,[1],4,1.0,6.0,3,11,4
4,ID_8PEN,ID_I1L9,[1],[],5.0,0,2,2,0,200.0,[1],2,11.0,6.0,2,11,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149,ID_F7X4,NaN,[1],[],2.0,1,2,3,0,30.0,[1],not mapped,NaN,NaN,1,5,3
150,ID_E1LI,ID_I1L9,"[3, 2]",[14],7.0,1,2,3,0,50.0,[2],not mapped,NaN,NaN,3,2,3
151,ID_MPSN,NaN,[1],[9],2.0,1,2,3,0,10.0,"[1, 5]",not mapped,NaN,NaN,2,2,7
152,ID_Z5QP,ID_5OWN,[1],[],31.0,0,2,2,1,NaN,[1],3,6.0,5.0,3,6,5


In [9]:
discussions_df

,FeatureF,DiscDate Year,DiscDate Month,DiscDate Day_of_week,DiscID,UserID
0,1,3,10,1,ID_Z77ETQ,ID_F2757IAI
1,0,3,12,3,ID_E47JKY,ID_F2757IAI
2,1,3,1,3,ID_CB4Y0N,ID_F2757IAI
3,0,3,9,7,ID_BNIHCF,ID_F2757IAI
4,1,3,5,7,ID_MLPYCO,ID_F2757IAI
...,...,...,...,...,...,...
6206,0,3,7,1,ID_07HHT5,ID_E2Q1K4TQ
6207,1,3,7,2,ID_9TID7A,ID_8I5VPQIF
6208,0,3,7,5,ID_IMGAT1,ID_UC2B2DBT
6209,1,3,11,2,ID_W3CY00,ID_VVUWHX7W


In [10]:
submissions_df

,UserID,FeatureG,CompID,SubDate Year,SubDate Month,SubDate Day_of_week
0,ID_8JP75F20,1,ID_GFDE,3,3,2
1,ID_8JP75F20,1,ID_GFDE,3,3,2
2,ID_8JP75F20,1,ID_GFDE,3,3,4
3,ID_8JP75F20,1,ID_GFDE,3,3,2
4,ID_8JP75F20,1,ID_GFDE,3,3,4
...,...,...,...,...,...,...
375758,ID_CX5N3Q88,1,ID_EZD0,3,11,3
375759,ID_CX5N3Q88,1,ID_EZD0,3,11,3
375760,ID_CX5N3Q88,1,ID_EZD0,3,11,3
375761,ID_J6MM98N2,1,ID_92AG,3,12,4


In [11]:
users_df

,UserID,FeatureX,Country,FeatureY,Points,UserDate Year,UserDate Month,UserDate Day_of_week
0,ID_N5LTBAPU,0,ID_DMRM,1,group 3,2,4,4
1,ID_CLSFQB0S,0,ID_Q02,3,group 3,1,5,4
2,ID_RE6T58Y4,0,ID_Q02,0,group 3,2,12,3
3,ID_XJQQRJV3,0,ID_Z8BI,0,group 3,2,9,2
4,ID_1JHU6A8S,0,ID_Q02,3,group 3,2,10,1
...,...,...,...,...,...,...,...,...
22402,ID_D4SARSC7,0,ID_5OWN,1,group 3,1,5,3
22403,ID_B8VJJMWK,0,ID_Q02,3,group 3,2,3,4
22404,ID_XAQGPGAZ,0,ID_Q02,3,group 3,2,3,1
22405,ID_1AO7PVP2,0,ID_Q02,3,group 3,2,5,5


In [12]:
train_df

,User_ID,month,year,CompPart,Comment,Sub,Disc,Target
0,ID_XI7BAR4Y,8,3,0,0,0,0,0
1,ID_XI7BAR4Y,8,2,0,0,0,0,0
2,ID_XI7BAR4Y,9,2,0,0,0,0,0
3,ID_XI7BAR4Y,9,3,0,0,0,0,0
4,ID_XI7BAR4Y,10,3,0,0,0,0,0
...,...,...,...,...,...,...,...,...
259827,ID_MAP5X6D4,12,3,0,0,0,0,0
259828,ID_QHUAHU76,12,3,0,0,0,0,0
259829,ID_8IKU22O5,12,3,1,0,0,0,1
259830,ID_NHWCR1IY,12,3,0,0,0,0,0


In [13]:
train_df

,User_ID,month,year,CompPart,Comment,Sub,Disc,Target
0,ID_XI7BAR4Y,8,3,0,0,0,0,0
1,ID_XI7BAR4Y,8,2,0,0,0,0,0
2,ID_XI7BAR4Y,9,2,0,0,0,0,0
3,ID_XI7BAR4Y,9,3,0,0,0,0,0
4,ID_XI7BAR4Y,10,3,0,0,0,0,0
...,...,...,...,...,...,...,...,...
259827,ID_MAP5X6D4,12,3,0,0,0,0,0
259828,ID_QHUAHU76,12,3,0,0,0,0,0
259829,ID_8IKU22O5,12,3,1,0,0,0,1
259830,ID_NHWCR1IY,12,3,0,0,0,0,0


In [14]:
test_df

,User_ID,month,year
0,ID_H1ELY25E,1,4
1,ID_H1ELY25E,2,4
2,ID_H1ELY25E,3,4
3,ID_463Q2BCO,1,4
4,ID_463Q2BCO,2,4
...,...,...,...
65218,ID_4XKWR8UN,3,4
65219,ID_L54061S5,3,4
65220,ID_I3C1N5RO,3,4
65221,ID_WWNR6I15,3,4


In [15]:
competition_participation_df

,CompID,UserID,PublicRank,Successful Submission Count,CompPartCreated Year,CompPartCreated Month,CompPartCreated Day_of_week
0,ID_WMUF,ID_UWBBZ9OF,rank 11,count 10,3,1,4
1,ID_MPSN,ID_UWBBZ9OF,NaN,NaN,3,1,7
2,ID_WMUF,ID_1N5J2PGO,NaN,count 10,3,1,5
3,ID_7MLO,ID_1N5J2PGO,NaN,NaN,2,10,5
4,ID_2KEY,ID_1N5J2PGO,NaN,NaN,2,5,4
...,...,...,...,...,...,...,...
48560,ID_XYJZ,ID_5C4D0VO2,rank 11,count 10,3,7,6
48561,ID_XYJZ,ID_JRJZQB8S,rank 11,count 10,3,7,5
48562,ID_XYJZ,ID_H4FM7RBV,NaN,NaN,3,7,5
48563,ID_XYJZ,ID_C3QOEMU4,rank 11,count 10,3,7,5


In [16]:
train_df.shape

(259832, 8)

In [17]:
competition_participation_df.shape

(48565, 7)

# Feature engineering

In [18]:
train_df

,User_ID,month,year,CompPart,Comment,Sub,Disc,Target
0,ID_XI7BAR4Y,8,3,0,0,0,0,0
1,ID_XI7BAR4Y,8,2,0,0,0,0,0
2,ID_XI7BAR4Y,9,2,0,0,0,0,0
3,ID_XI7BAR4Y,9,3,0,0,0,0,0
4,ID_XI7BAR4Y,10,3,0,0,0,0,0
...,...,...,...,...,...,...,...,...
259827,ID_MAP5X6D4,12,3,0,0,0,0,0
259828,ID_QHUAHU76,12,3,0,0,0,0,0
259829,ID_8IKU22O5,12,3,1,0,0,0,1
259830,ID_NHWCR1IY,12,3,0,0,0,0,0


In [19]:
new_df = train_df.sort_values(["User_ID", "year", "month"], ascending=[True, True, True])
new_df

,User_ID,month,year,CompPart,Comment,Sub,Disc,Target
100500,ID_000VV0KM,12,2,1,0,1,0,1
100501,ID_000VV0KM,1,3,0,0,0,0,0
100502,ID_000VV0KM,2,3,0,0,0,0,0
100503,ID_000VV0KM,3,3,0,0,0,0,0
100504,ID_000VV0KM,4,3,0,0,0,0,0
...,...,...,...,...,...,...,...,...
219761,ID_ZZXDLYXB,8,3,0,0,0,0,0
219762,ID_ZZXDLYXB,9,3,0,0,0,0,0
219763,ID_ZZXDLYXB,10,3,0,0,0,0,0
219764,ID_ZZXDLYXB,11,3,0,0,0,0,0


## 0. Let's create the column of total months:

In [20]:
#new_df["total_month"] = new_df[["month", "year"]] - new_df.groupby("User_ID").first()[["month", "year"]].values
#new_df

In [21]:
new_df["total_month"] = new_df["month"] + new_df["year"] * 12
new_df["total_month"] = new_df.groupby("User_ID")["total_month"].diff().fillna(0)
new_df

,User_ID,month,year,CompPart,Comment,Sub,Disc,Target,total_month
100500,ID_000VV0KM,12,2,1,0,1,0,1,0.0
100501,ID_000VV0KM,1,3,0,0,0,0,0,1.0
100502,ID_000VV0KM,2,3,0,0,0,0,0,1.0
100503,ID_000VV0KM,3,3,0,0,0,0,0,1.0
100504,ID_000VV0KM,4,3,0,0,0,0,0,1.0
...,...,...,...,...,...,...,...,...,...
219761,ID_ZZXDLYXB,8,3,0,0,0,0,0,1.0
219762,ID_ZZXDLYXB,9,3,0,0,0,0,0,1.0
219763,ID_ZZXDLYXB,10,3,0,0,0,0,0,1.0
219764,ID_ZZXDLYXB,11,3,0,0,0,0,0,1.0


## 1. The number of activities by a user in the previous month

In [22]:
train_df.groupby(["User_ID", "year", "month"])[["CompPart", "Comment",	"Sub", "Disc"]].sum()

CompPart  Comment  Sub  Disc
User_ID     year month                              
ID_000VV0KM 2    12            1        0    1     0
            3    1             0        0    0     0
                 2             0        0    0     0
                 3             0        0    0     0
                 4             0        0    0     0
...                          ...      ...  ...   ...
ID_ZZXDLYXB 3    8             0        0    0     0
                 9             0        0    0     0
                 10            0        0    0     0
                 11            0        0    0     0
                 12            0        0    0     0

[259832 rows x 4 columns]

In [23]:
new_df["prev_m_act"] = new_df.groupby('User_ID')[["CompPart", "Comment", "Sub", "Disc"]].shift(1).values.sum(axis=1)
new_df = new_df.fillna(0.0)
new_df.head(40)

,User_ID,month,year,CompPart,Comment,Sub,Disc,Target,total_month,prev_m_act
100500,ID_000VV0KM,12,2,1,0,1,0,1,0.0,0.0
100501,ID_000VV0KM,1,3,0,0,0,0,0,1.0,2.0
100502,ID_000VV0KM,2,3,0,0,0,0,0,1.0,0.0
100503,ID_000VV0KM,3,3,0,0,0,0,0,1.0,0.0
100504,ID_000VV0KM,4,3,0,0,0,0,0,1.0,0.0
100505,ID_000VV0KM,5,3,0,0,0,0,0,1.0,0.0
100506,ID_000VV0KM,6,3,0,0,0,0,0,1.0,0.0
100507,ID_000VV0KM,7,3,0,0,0,0,0,1.0,0.0
100495,ID_000VV0KM,8,3,0,0,0,0,0,1.0,0.0
100496,ID_000VV0KM,9,3,0,0,0,0,0,1.0,0.0


## 2. The number of months since the last activity by the user.

In [24]:
'''
def months_since_last_act(group):
    
    s = []
    a = 0
    for i, row in group.iterrows():
        if row.iloc[7] == 1:
            s.append(0)
            a = 0
        else:
            a += 1
            s.append(a)
    
    s = pd.DataFrame(s, columns=["months_since_last_act"])
    s = s.reset_index()
    s = s.explode(column="months_since_last_act")
    s = s.reset_index().drop("index", axis=1)

    return s
        
col_df = new_df.groupby('User_ID').apply(months_since_last_act)
col_df
'''

'\ndef months_since_last_act(group):\n    \n    s = []\n    a = 0\n    for i, row in group.iterrows():\n        if row.iloc[7] == 1:\n            s.append(0)\n            a = 0\n        else:\n            a += 1\n            s.append(a)\n    \n    s = pd.DataFrame(s, columns=["months_since_last_act"])\n    s = s.reset_index()\n    s = s.explode(column="months_since_last_act")\n    s = s.reset_index().drop("index", axis=1)\n\n    return s\n        \ncol_df = new_df.groupby(\'User_ID\').apply(months_since_last_act)\ncol_df\n'

In [25]:
def my_function(group):
    s = []
    a = 0
    for i, row in group.iterrows():
        if row.iloc[7] == 1:
            s.append(0)
            a = 0
        else:
            a += 1
            s.append(a)

    
    s = pd.DataFrame(s, columns=["months_since_last_act"])
    return s
        
col = new_df.groupby('User_ID').apply(my_function)
col

months_since_last_act
User_ID                             
ID_000VV0KM 0                      0
            1                      1
            2                      2
            3                      3
            4                      4
...                              ...
ID_ZZXDLYXB 4                      4
            5                      5
            6                      6
            7                      7
            8                      8

[259832 rows x 1 columns]

In [26]:
col_df = pd.DataFrame(col, columns=["months_since_last_act"])
col_df = col_df.reset_index()
col_df

,User_ID,level_1,months_since_last_act
0,ID_000VV0KM,0,0
1,ID_000VV0KM,1,1
2,ID_000VV0KM,2,2
3,ID_000VV0KM,3,3
4,ID_000VV0KM,4,4
...,...,...,...
259827,ID_ZZXDLYXB,4,4
259828,ID_ZZXDLYXB,5,5
259829,ID_ZZXDLYXB,6,6
259830,ID_ZZXDLYXB,7,7


In [27]:
col_df = col_df.explode(column="months_since_last_act")
col_df = col_df.reset_index().drop("index", axis=1)
col_df

,User_ID,level_1,months_since_last_act
0,ID_000VV0KM,0,0
1,ID_000VV0KM,1,1
2,ID_000VV0KM,2,2
3,ID_000VV0KM,3,3
4,ID_000VV0KM,4,4
...,...,...,...
259827,ID_ZZXDLYXB,4,4
259828,ID_ZZXDLYXB,5,5
259829,ID_ZZXDLYXB,6,6
259830,ID_ZZXDLYXB,7,7


In [28]:
new_df = pd.concat([new_df.reset_index(), col_df.iloc[:, -1].reset_index()], axis=1).drop("index", axis=1)
new_df = new_df.rename(columns={"User_ID": "UserID"})
new_df

,UserID,month,year,CompPart,Comment,Sub,Disc,Target,total_month,prev_m_act,months_since_last_act
0,ID_000VV0KM,12,2,1,0,1,0,1,0.0,0.0,0
1,ID_000VV0KM,1,3,0,0,0,0,0,1.0,2.0,1
2,ID_000VV0KM,2,3,0,0,0,0,0,1.0,0.0,2
3,ID_000VV0KM,3,3,0,0,0,0,0,1.0,0.0,3
4,ID_000VV0KM,4,3,0,0,0,0,0,1.0,0.0,4
...,...,...,...,...,...,...,...,...,...,...,...
259827,ID_ZZXDLYXB,8,3,0,0,0,0,0,1.0,0.0,4
259828,ID_ZZXDLYXB,9,3,0,0,0,0,0,1.0,0.0,5
259829,ID_ZZXDLYXB,10,3,0,0,0,0,0,1.0,0.0,6
259830,ID_ZZXDLYXB,11,3,0,0,0,0,0,1.0,0.0,7


In [29]:
#new_df.head(40)

## Eliminating wrong target values

In [30]:
def fix_incorrect_target_labels(train):

    check1 = train[train['Comment']==1]['UserID'].unique()
    check2 = comments_df[comments_df['UserID'].isin(check1)]['UserID'].unique()
    
    # TO BE CORRECTED USERIDS FOR COMMENTS TABLE
    tbc = list(set(check1) - set(check2))

    for userid in tbc:
        train.loc[train['UserID']==userid, 'Comment'] = 0

    check1 = train[train['Comment']==1]['UserID'].unique()
    check2 = comments_df[comments_df['UserID'].isin(check1)]['UserID'].unique()
    
    # SHOULD BE CORRECTED NOW: LETS CHECK
    tbc = list(set(check1) - set(check2))
    
    check1 = train[train['CompPart']==1]['UserID'].unique()
    check2 = competition_participation_df[competition_participation_df['UserID'].isin(check1)]['UserID'].unique()
    tbc = list(set(check1) - set(check2))

    for userid in tbc:
        train.loc[train['UserID']==userid, 'CompPart'] = 0

    check1 = train[train['CompPart']==1]['UserID'].unique()
    check2 = competition_participation_df[competition_participation_df['UserID'].isin(check1)]['UserID'].unique()
    tbc = list(set(check1) - set(check2))

    train['Target'] = train['CompPart'] + train['Comment'] + train['Sub'] + train['Disc']
    train.loc[train['Target']>0, 'Target'] = 1
    return train

train = fix_incorrect_target_labels(new_df)
train

14853
14850


,UserID,month,year,CompPart,Comment,Sub,Disc,Target,total_month,prev_m_act,months_since_last_act
0,ID_000VV0KM,12,2,1,0,1,0,1,0.0,0.0,0
1,ID_000VV0KM,1,3,0,0,0,0,0,1.0,2.0,1
2,ID_000VV0KM,2,3,0,0,0,0,0,1.0,0.0,2
3,ID_000VV0KM,3,3,0,0,0,0,0,1.0,0.0,3
4,ID_000VV0KM,4,3,0,0,0,0,0,1.0,0.0,4
...,...,...,...,...,...,...,...,...,...,...,...
259827,ID_ZZXDLYXB,8,3,0,0,0,0,0,1.0,0.0,4
259828,ID_ZZXDLYXB,9,3,0,0,0,0,0,1.0,0.0,5
259829,ID_ZZXDLYXB,10,3,0,0,0,0,0,1.0,0.0,6
259830,ID_ZZXDLYXB,11,3,0,0,0,0,0,1.0,0.0,7


## Visualisation

In [31]:
train.head(40)

,UserID,month,year,CompPart,Comment,Sub,Disc,Target,total_month,prev_m_act,months_since_last_act
0,ID_000VV0KM,12,2,1,0,1,0,1,0.0,0.0,0
1,ID_000VV0KM,1,3,0,0,0,0,0,1.0,2.0,1
2,ID_000VV0KM,2,3,0,0,0,0,0,1.0,0.0,2
3,ID_000VV0KM,3,3,0,0,0,0,0,1.0,0.0,3
4,ID_000VV0KM,4,3,0,0,0,0,0,1.0,0.0,4
5,ID_000VV0KM,5,3,0,0,0,0,0,1.0,0.0,5
6,ID_000VV0KM,6,3,0,0,0,0,0,1.0,0.0,6
7,ID_000VV0KM,7,3,0,0,0,0,0,1.0,0.0,7
8,ID_000VV0KM,8,3,0,0,0,0,0,1.0,0.0,8
9,ID_000VV0KM,9,3,0,0,0,0,0,1.0,0.0,9
